# IMPORTS

In [1]:
from datetime import datetime

from clientes.analisis_fuensalida import analisis_fuensalida_params as params
from read_data_base.data_collectors import SQLServerCollector
from read_data_base.softland_querys import QueryTipoDocumentos,QueryAuxiliares,QueryTodosMovimientos,QueryTodosComprobantes

from analisis_cuentas.informe_cuentas_softland import remove_initial_movements

import pandas as pd

# GET DATA

In [2]:
input_date = datetime.strptime("30/09/2021","%d/%m/%Y")

collector = SQLServerCollector( params.server,
                                params.port,
                                params.database,
                                params.username,
                                params.password)

docs_query = QueryTipoDocumentos()
auxiliares_query = QueryAuxiliares()
movimientos_query = QueryTodosMovimientos(params.reembolsos)
comprobantes_query = QueryTodosComprobantes()

#Connect to database
collector.connect()

#Read Doc Types
all_docs_df = collector.get_data(docs_query)

#Read Auxiliares
auxiliares = collector.get_data(auxiliares_query)

#Read comprobantes
comprobantes = collector.get_data(comprobantes_query)

#Read Movements
movimientos = collector.get_data(movimientos_query)

# TESTING AREA

In [16]:
# clean_movements = remove_initial_movements(movimientos)
clean_movements = movimientos[movimientos.MovGlosa.str.contains('Gasto|gasto|reembolso|Reembolso')]
# clean_movements = movimientos[movimientos.TipDocCb != '00']
# filtered_movements = clean_movements.groupby(['CodAux']).filter(lambda group: group.Debe.sum()-group.Haber.sum()!=0).sort_values(['CodAux','CpbAno','CpbNum'])
# # clean_movements.set_index(['CodAux','CpbAno','CpbNum'])
clean_movements.to_excel('movimientors_reembolsos.xlsx')

In [21]:
comprobantes[(comprobantes.CpbAno == '2017')&((comprobantes.CpbNum == '00002070')|(comprobantes.CpbNum == '00002081'))].to_excel('cpb.xlsx')

In [3]:
pd.merge(
        movimientos,
        comprobantes,
        how="left",
        left_on=['CpbAno','CpbNum'],
        right_on=['CpbAno','CpbNum'],
        suffixes=("", "_")).to_excel('movimientors_reembolsos.xlsx')

In [31]:
output_df = pd.DataFrame([[0,1,2,3],[0,1,2,3],[0,1,2,3]])
output_df.iloc[1,:] = "30/09/2021"
output_df

,0,1,2,3
0,0,1,2,3
1,30/09/2021,30/09/2021,30/09/2021,30/09/2021
2,0,1,2,3
